<a href="https://colab.research.google.com/github/evinracher/3008410-intelligent-systems/blob/main/week2/exercise1/Post_Training_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install transformers==4.40.2 --force-reinstall

  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
Using cached transformers-4.40.2-py3-none-any.whl (9.0 MB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.7 MB 6.1 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.7 MB 6.0 MB/s eta 0:00:02
   ------------ ---------------------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.


In [ ]:
import sys
!{sys.executable} -m pip install "accelerate>=0.21.0" --upgrade

In [ ]:
import transformers
print(transformers.__version__)

4.40.2


In [ ]:
from transformers import (
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from datasets import Dataset
import evaluate
import numpy as np
import os
import pandas as pd

# --- Load dataset ---
folder_path = r"C:/Users/John/Downloads/Universidad/AI Generativa Course/Exercises/Exercise2"
dataset_name = "eng_small.csv"
path = os.path.join(folder_path, dataset_name)
data = Dataset.from_csv(path, encoding='utf-8')
data = data.train_test_split(test_size=0.1)

# --- Tokenizer & Model ---
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # PyTorch version

# --- Preprocessing ---
prefix = "translate: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = data.map(preprocess_function, batched=True, remove_columns=["engl", "spa"])

# --- Data Collator ---
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# --- Metrics (BLEU) ---
bleu = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    print("Preds shape:", preds.shape)
    print("Labels shape:", labels.shape)

    if isinstance(preds, tuple):
        preds = preds[0]

    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    try:
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    except Exception as e:
        print("Decoding error:", e)
        decoded_preds = [""] * len(preds)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Remove empty predictions or references
    filtered_preds = []
    filtered_refs = []
    for pred, ref in zip(decoded_preds, decoded_labels):
        if pred.strip() and ref.strip():
            filtered_preds.append(pred)
            filtered_refs.append([ref])

    if not filtered_preds:
        print("No valid predictions to evaluate.")
        return {"bleu": 0.0}

    result = bleu.compute(predictions=filtered_preds, references=filtered_refs)
    return {"bleu": result["bleu"]}


# --- Training arguments ---
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate=True,
    generation_max_length=128,  # ← Added to avoid generation warnings
    logging_dir="./logs",
    report_to="none"
)

# --- Trainer ---
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# --- Train ---
trainer.train()


C:\Users\John\miniconda312\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/18096 [00:00<?, ? examples/s]

Map:   0%|          | 0/2011 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu
1,1.944600,1.503251,0.170398
2,1.561900,1.291170,0.239736
3,1.403500,1.236555,0.258707


C:\Users\John\miniconda312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

C:\Users\John\miniconda312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Preds shape: (2011, 24)
Labels shape: (2011, 39)


C:\Users\John\miniconda312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Preds shape: (2011, 115)
Labels shape: (2011, 39)


TrainOutput(global_step=3393, training_loss=1.7169003991155964, metrics={'train_runtime': 4277.1433, 'train_samples_per_second': 12.693, 'train_steps_per_second': 0.793, 'total_flos': 153464914575360.0, 'train_loss': 1.7169003991155964, 'epoch': 3.0})

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned model
model_fp32 = AutoModelForSeq2SeqLM.from_pretrained("./results/trained_model")
tokenizer = AutoTokenizer.from_pretrained("./results/trained_model")

# Apply dynamic quantization
model_quantized = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)

# Save quantized model state_dict
torch.save(model_quantized.state_dict(), "./results/quantized_model/quantized_model_state_dict.pt")

# Save model config and tokenizer for later reload
model_quantized.config.save_pretrained("./results/quantized_model")
tokenizer.save_pretrained("./results/quantized_model")

print("✅ Quantized model and tokenizer saved.")


✅ Quantized model and tokenizer saved.


Compare Size of the models

In [ ]:
import os

def get_file_size(path):
    size = os.path.getsize(path) / (1024 ** 2)  # Convert bytes to MB
    print(f"{path} size: {size:.2f} MB")
    return size

# Original model (multiple PyTorch weights shards or single file)
original_model_path = "./results/trained_model/model.safetensors"
quantized_model_path = "./results/quantized_model/quantized_model.pt"

get_file_size(original_model_path)
get_file_size(quantized_model_path)


./results/trained_model/model.safetensors size: 230.83 MB
./results/quantized_model/quantized_model.pt size: 120.64 MB


120.63667583465576

Compare inferences and time

In [ ]:
import torch
import time
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cpu")

# Load model and tokenizer
model_fp32 = AutoModelForSeq2SeqLM.from_pretrained("./results/trained_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("./results/trained_model")
model_fp32.eval()

# Quantize dynamically in-memory
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)
model_int8.eval()

# Inference input
text = "it is good to have friends"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Inference function
def run_inference(model, label):
    start = time.time()
    with torch.no_grad():
        output = model.generate(**inputs, max_length=28)
    elapsed = time.time() - start
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"{label} Time: {elapsed:.3f}s | Output: {decoded}")
    return decoded, elapsed

print("\n Comparing Inference...")
out_fp32, t_fp32 = run_inference(model_fp32, "FP32")
out_int8, t_int8 = run_inference(model_int8, "INT8 Quantized")

# Optional comparison summary
print("\n Output Match" if out_fp32 == out_int8 else "\n Output Slightly Differ")
print(f"\n FP32 Time: {t_fp32:.3f}s | INT8 Time: {t_int8:.3f}s")



 Comparing Inference...
FP32 Time: 0.217s | Output: e bu bu bu buen a amigos.
INT8 Quantized Time: 0.205s | Output: b bu bu bu bu a amigos.

 Output Slightly Differ

 FP32 Time: 0.217s | INT8 Time: 0.205s


## Exercise

- Quantize the model to fp16 and write the new size of the model